# Variables

## Constructors

A variable is a tensor that has the property of maintaining its value across multiple calls to `run()`. A variable is added to a computational graph wiht one of the following constructors.

1. `tf.Variable`
2. `tf.get_variable`

### tf.Variable

The `tf.Variable` constructor requires an initial value, i.e. a tensor of any type and shape. The initial value defines the type and shape. After the construction the type and shape are fixed. The values can be modified through the assignment methods. It is also possible to modify the shape via assignments, as shown below.

In the example below we add a variable to the graph, and use the `global_variables_initializer()` to initialize its value.

In [1]:
import tensorflow as tf

tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    u = tf.Variable(13.0, name='u')
    init = tf.global_variables_initializer()
    
with tf.Session(graph=graph) as sess:
    sess.run(init)
    u_val = sess.run(u)
print(u_val)

/home/giovenko/miniconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


13.0


The type of the variable is deducted from the initial value. We can initialize the variable individually with `sess.run(u.initializer)`.

In [4]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    u = tf.Variable(13.0, name='u')

with tf.Session(graph=graph) as sess:
    sess.run(u.initializer)  # We have only one variable. We can initialize it individually.
    u_val = sess.run(u)
print(u_val)

13.0


Initialization can be performed with an assignment operator.

In [6]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    u = tf.Variable(13.0, name='u')

with tf.Session(graph=graph) as sess:
    u = tf.assign(ref=u, value=13.0)  # Is equivalent to an initialization.
    u_val = sess.run(u)
print(u_val)

13.0


Assignment can also change the shape of a variable. For this to work, one mulst pass the argument `validate_shape=False`.

In [7]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    u = tf.Variable(13.0, name='u')

with tf.Session(graph=graph) as sess:
    u = tf.assign(ref=u, value=[[1.0], [2.0], [3.0]], validate_shape=False)
    u_val = sess.run(u)
print(u_val)

[[1.]
 [2.]
 [3.]]


Another form of initialization is by restoring a saved model. It can happen that one variable needs to be initialized based on the value of another variable. You can use `initialized_value()` for this purpose.

In [8]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    u = tf.Variable(13.0, name='u')
    v = tf.Variable(u.initialized_value() + 2.0, name='v')  # Use the value of u to initialize
    init = tf.global_variables_initializer()

with tf.Session(graph=graph) as sess:
    sess.run(init)
    v_val = sess.run(v)
print(v_val)

15.0


One can find which variables are not initialized with `tf.report_uninitialized_variables()` or test whether a given variable is initialized or not via `tf.is_variable_initialized()`.

## Variable Collections

All variables in a graph are automatically collected in **graph collections**. By default the `tf.Variable` constructor adds new variables to `GraphKeys.GLOBAL_VARIABLES`. You can inspect the content of this collections in (at least) two ways:

1. with the `tf.global_variables()` function.
2. with the `tf.GraphKeys` collections.

In [22]:
# Show the global variables in the graph
with graph.as_default():
    print(tf.global_variables())

[<tf.Variable 'u:0' shape=() dtype=float32_ref>, <tf.Variable 'v:0' shape=() dtype=float32_ref>]


Variables are added to the graph collections listed in `collections`, which defaults to `GraphKeys.GLOBAL_VARIABLES`. Note that the global initializer initializes all the variables in this collection. If the `trainable` argument in the constructor is set to `True` (the default0, the variables are also added to `GraphKeys.TRAINABLE_VARIABLES`. Therefore every `tf.Variable` is by default placed in the two collections: `GraphKeys.GLOBAL_VARIABLES` and `GraphKeys.TRAINABLE_VARIABLES`
One can specify to which collections the variable will be added with the `collections` argument in the `tf.Variable` constructor.

In [28]:
with graph.as_default():
    print(graph.collections)
    print([tf.GraphKeys.TRAINABLE_VARIABLES, 
           tf.GraphKeys.GLOBAL_VARIABLES, 
           tf.GraphKeys.COND_CONTEXT])

['trainable_variables', 'variables', 'cond_context']
['trainable_variables', 'variables', 'cond_context']


## tf.get_variable

Using [tf.get_variable](https://www.tensorflow.org/api_docs/python/tf/get_variable) is the preferred way to create a variable. This function prefixes the variable name with the variable scope and performs reuse checks. To create a variable, we pass the name and the shape. The dtype will be `tf.float32` by default. This can be changed with the `dtype` argument. The initializer is `None` by default, in which case the initializer of the variable scope is used. If this too is `None`, a `glorot_uniform_initializer` is used. The initializer can be a tensor, in which case `shape` should not be specified.

In [30]:
tf.reset_default_graph()

graph = tf.Graph()
with graph.as_default():
    u = tf.get_variable(name='u', shape=[], dtype=tf.int32)
    u = tf.assign(ref=u, value=5)  # Initialization by assignment.

with tf.Session(graph=graph) as sess:
    u_val = u.eval()
print(u_val)

5


One can create new collections and add variables to them by using `tf.add_to_collection('collection_name', var_name)`. To see the variables in a collection, use `tf.get_collection()`.

### tf.control_dependiencies and read_value

Variables can be modified in the course of session runs. In some cases we may want to be sure about what value the variable has. We can do this by using a `tf.control_dependencies` context manager, and `read_value`. The example below is taken from this part of the [documentation](https://www.tensorflow.org/programmers_guide/variables)

In [31]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
  w = v.read_value()  # w is guaranteed to reflect v's value after the
                      # assign_add operation.

Variables can be partitioned for sharding across devices. There are specific functions for this purpose. Look for "Variable Partitioners for Sharding" in the [variables documentation](https://www.tensorflow.org/api_guides/python/state_ops#Variable_Partitioners_for_Sharding)

### More on graphs and collections

Let's consider the case below, where we create a graph, and add a constant tensor.

In [33]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    c = tf.constant(7.0, name='c')

with tf.Session(graph=graph) as sess:
    c_val = sess.run(c)
print(c_val)

7.0


The graph does not contain variables, therefore all the collections are empty.

In [39]:
with graph.as_default():
    print(graph.collections)
    graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[]


We can retrieve the constant tensor by name. This will be `c:0`.

In [40]:
with graph.as_default():
    print(graph.get_tensor_by_name('c:0'))

Tensor("c:0", shape=(), dtype=float32)


Let's add two variables to the graph: one trainable and one not.

In [41]:
with graph.as_default():
    u = tf.get_variable(name='u', shape=[])
    v = tf.get_variable(name='v', shape=[], trainable=False)
    init = tf.global_variables_initializer()

Now the collections of trainable and non trainable variables should be populated.

In [45]:
with graph.as_default():
    print(graph.collections)
    print(graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
    print(graph.get_collection(tf.GraphKeys.VARIABLES))

[('__varscope',), ('__variable_store',), 'trainable_variables', 'variables']
[<tf.Variable 'u:0' shape=() dtype=float32_ref>]
[<tf.Variable 'u:0' shape=() dtype=float32_ref>, <tf.Variable 'v:0' shape=() dtype=float32_ref>]
